In [1]:
#42/42

In [1]:
import yaml
import pandas as pd
import json
import csv
import urllib.request as urlr

In [2]:
# Global configs #
pd.options.display.max_rows = 100
pd.options.display.max_colwidth = 1000

In [26]:
# Global vars #

with open('/home/jovyan/work/files/AV_api_key.yaml', 'r') as file:
    file_data = yaml.safe_load(file)
AV_API_KEY = file_data['api_key']

TIME_SERIES = "TIME_SERIES_DAILY"

# This is a list of a selected group Franklin Templeton's ETFs,
# each associated with a specific country or region.
# For clarification:
# ETF symbol | Country/region    
COUNTRY_SYMBOL = {
    'FLFR' : 'France',
    'FLGB' : 'UK',
    'FLGR' : 'Germany',
    'FLIY' : 'Italy',
    'FLSW' : 'Switzerland',
    'FLCA' : 'Canada',
    'FLJP' : 'Japan',
    'FLKR' : 'South Korea',
    'FLAU' : 'Australia',
    'FLTW' : 'Taiwan',
    'FLMX' : 'Mexico',
    'FLSA' : 'Saudi Arabia',
    'FLZA' : 'South Africa',
    'FLBR' : 'Brazil',
    'FLIN' : 'India',
    'FLCH' : 'China',
    'FLEU' : 'Eurozone',
    'FLEE' : 'Europe',
    'FLLA' : 'Latin America',
    'FLAX' : 'Asian excl. Japan'
}
SYMBOL_LIST = COUNTRY_SYMBOL.keys()

In [4]:
# Note: since API's tend to have a limited number of calls, it's probably best to store the read data into a file.

In [27]:
df_av_data = pd.DataFrame()
for symbol in SYMBOL_LIST:
    # Get the data from the AV API #
    url = "https://www.alphavantage.co/query?function=" + TIME_SERIES + "&symbol=" + symbol + "&outputsize=full&apikey=" + AV_API_KEY
    #with open('/home/jovyan/work/code/myjson.json', 'r') as file:
    av_raw_data = json.load(urlr.urlopen(url))

    stock_symbol = av_raw_data['Meta Data']['2. Symbol']
    df_av_symbol_data = pd.DataFrame.from_dict(av_raw_data['Time Series (Daily)']).T
    df_av_symbol_data['stock_symbol'] = stock_symbol
    #print(df_av_symbol_data)
    df_av_data = pd.concat([df_av_data, df_av_symbol_data])
    print(df_av_data.shape)

(1513, 6)
(3475, 6)
(5437, 6)
(6950, 6)
(8848, 6)
(10810, 6)
(12772, 6)
(14734, 6)
(16695, 6)
(18644, 6)
(20606, 6)
(22334, 6)
(23613, 6)
(25575, 6)
(27473, 6)
(29435, 6)
(31397, 6)
(33359, 6)
(35087, 6)
(36985, 6)


In [28]:
# Feature engineering #

# Renaming columns
df_av_data.rename(columns={"1. open" : "open",
                          "2. high" : "high",
                          "3. low" : "low",
                          "4. close" : "close",
                          "5. volume" : "volume"}, inplace=True)

In [29]:
# Reorder columns
cols = df_av_data.columns.to_list()
cols = cols[-1:] + cols[:-1] # Put the symbol to the front
df_av_data = df_av_data[cols]

In [30]:
df_av_data

,stock_symbol,open,high,low,close,volume
2023-11-09,FLFR,27.7300,27.7600,27.7299,27.7299,924
2023-11-08,FLFR,27.7203,27.7203,27.7203,27.7203,48
2023-11-07,FLFR,27.7386,27.7386,27.7102,27.7102,282
2023-11-06,FLFR,27.7103,27.7600,27.6900,27.7202,1469
2023-11-03,FLFR,26.3500,27.7200,25.0500,27.6852,1400
...,...,...,...,...,...,...
2018-02-14,FLAX,23.8600,23.8600,23.8600,23.8600,200
2018-02-13,FLAX,23.6800,23.6800,23.6800,23.6800,900
2018-02-12,FLAX,23.3800,23.3800,23.3800,23.3800,105
2018-02-09,FLAX,22.9700,22.9700,22.9700,22.9700,1


In [31]:
# Store the data (frame) in a file

file_mode = ''
data_file_dir = '/home/jovyan/work/files/data/stocks_day.csv'
needs_header= ''
with open(data_file_dir, 'a+') as file:
    csv_reader = csv.reader(file)
    try:
        # CSV always has one line in it, so we check
        # for two to see if there's content (a header
        # in this case). If so, append the extra data
            file.seek(0)
            csv_reader.__next__()
            csv_reader.__next__()
            file_mode = 'a'
            needs_header = False
    except: # File doesn't have any content in it
            # so we write the header
            file_mode= 'w'
            needs_header = True

with open(data_file_dir, file_mode) as file:
    df_av_data.to_csv(file, index=True, index_label='date', header=needs_header)

In [35]:
# If data is requested again to get more up to date results, update the file
# by removing duplicates and reordering the data

with open(data_file_dir, 'r') as file:
    df = pd.read_csv(file)
    df.drop_duplicates(inplace=True)
    #df.sort_values(by=['stock_symbol', 'date'], ascending = [True, True], inplace=True) # Not needed for now

# Replace the file with itself having the clean and
# updated data
with open(data_file_dir, 'w') as file:
    df.to_csv(file, index=False, header=True)